In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
import torch.optim as optim

# Time parameters
T = 10.0
dt = 0.01
t = np.arange(0, T, dt)

# Nonlinear system: m*x'' + c(v)*v + k(x)*x = d(t)
def nonlinear_system(t, y, d_func):
    x, v = y
    d = d_func(t)
    m = 1.0
    c = 0.5 + 0.1 * v**2  # Nonlinear damping
    k = 2.0 + 0.5 * x**2  # Nonlinear spring
    dxdt = v
    dvdt = (d - c * v - k * x) / m
    return [dxdt, dvdt]

def generate_trajectory(d_func):
    sol = solve_ivp(fun=lambda t, y: nonlinear_system(t, y, d_func),
                    t_span=(0, T), y0=[0.0, 0.0], t_eval=t, method='RK45')
    return sol.y.T

# Generate dataset
n_samples = 200
X, Y = [], []
for _ in range(n_samples):
    u = np.random.randn(len(t)) * 0.5
    d_func = lambda t_val: np.interp(t_val, t, u)
    traj = generate_trajectory(d_func)
    X.append(u)
    Y.append(traj[:, 0])  # output: position x(t)

X = np.array(X)
Y = np.array(Y)

# Neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super(SimpleNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        return self.net(x)

# Train model
model = SimpleNN(len(t))
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

for epoch in range(500):
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, Y_tensor)
    loss.backward()
    optimizer.step()

print("Final training loss:", loss.item())

# Estimate worst-case gain of learned model
def empirical_gain(model, n_trials=500):
    max_gain = 0.0
    for _ in range(n_trials):
        d = np.random.randn(len(t))
        d_norm = np.linalg.norm(d)
        d_unit = d / (d_norm + 1e-8)
        with torch.no_grad():
            d_tensor = torch.tensor(d_unit[None, :], dtype=torch.float32)
            y = model(d_tensor).numpy().squeeze()
        gain = np.linalg.norm(y) / (np.linalg.norm(d_unit) + 1e-8)
        max_gain = max(max_gain, gain)
    return max_gain

gamma_empirical = empirical_gain(model)
print(f"Empirical worst-case gain of the learned model: {gamma_empirical:.3f}")


Final training loss: 1.5096187780727632e-05
Empirical worst-case gain of the learned model: 1.642
